In [1]:
import openpyxl
import nltk
import collections
import re

In [2]:
file = openpyxl.load_workbook('FILE_DIRECTORY')
clist = file.get_sheet_by_name("companies")
tags = file.get_sheet_by_name("Tags")
newly_added = file.get_sheet_by_name("newly_added")
################################################ LOAD SPREADSHEET ################################################
# i = 2 
# while i < clist.max_row:
#     ind = str(i)
#     name = 'B'
#     cInd = 'A'
#     name+=ind
#     cInd+=ind
#     print ("ID:", clist[cInd].value ,"          Name:", clist[name].value )
#     i+=1
###################################################################################################################

In [3]:
############################################## STORE CNAMES INTO DICT #############################################
cDict = collections.OrderedDict()
j = 2
while j <= clist.max_row:
    ind = str(j)
    cname = 'B'+ind
    cInd = 'A'+ind
    cDict[clist[cInd].value] = clist[cname].value
    j+=1
###################################################################################################################
############################################## STORE TAGS INTO DICT ###############################################
cTags = collections.OrderedDict()
p = 1
while p <= tags.max_row:
    ind = str(p)
    tag_name = 'B'+ind
    tag_id = 'A'+ind
    cTags[tags[tag_id].value] = tags[tag_name].value
    p+=1
###################################################################################################################
print(cTags[586])

FinTech


In [4]:
############################################## PROCESS CNAMES & GET URL ###########################################
cURL = collections.OrderedDict()

for cid, cname in cDict.items():
    cmp = cname.lower()
    cmp = cmp.replace(' ','-')
    cmp = cmp.replace('&','-')
    cmp = cmp.replace(',','')
    cmp = cmp.replace('.','')
    cmp = cmp.replace(' ','')
    url = 'https://www.crunchbase.com/organization/'+cmp+'#/entity'
    cURL[cid] = url
###################################################################################################################

In [8]:
def delete(row_num):
    clist['E'+str(row_num)] = 'X'
    print ("Row", row_num, "marked as delete\n")
def not_sure(row_num):
    clist['G'+str(row_num)] = 'X'
    print ("Row", row_num, "marked as not sure\n")
def other(row_num):
    clist['H'+str(row_num)] = 'X'
    print ("Row", row_num, "marked as other\n")
def new(Id, tag_id,inserted):
    newly_added['A'+str(inserted)] = Id
    newly_added['B'+str(inserted)] = cDict[Id]
    newly_added['C'+str(inserted)] = str(tag_id)
    newly_added['D'+str(inserted)] = cTags[int(tag_id)]
    newly_added['F'+str(inserted)] = 'X'
    print ("The tag: ", cTags[int(tag_id)], " was added for ", cDict[Id], " in row ", inserted)
def keep(row_num):
    #do nothing
    print ("Row", row_num, "was not changed")

process = {'d' : delete,
           'ns': not_sure,
           'o' : other,
           'n' : new,
           'k' : keep,
          }

In [14]:
############################################## PROCESS COMP #######################################################
def manageEntries(start_row):
    i = start_row
    inserted = 2
    while i <= clist.max_row:
        Id = clist['A'+str(i)].value
        print("Link to the crunchbase of:", cDict[Id], ":", cURL[Id])
        print("Current entries for", cDict[Id], ":\n")
        while clist['A'+str(i)].value==Id:
            if (clist['C'+str(i)].value<572 or clist['C'+str(i)].value>608):
                process[d](i)
                print("This entry, with tag:", clist['D'+str(i)].value, "was automatically deleted")
            else:
                print("Row:", i, "             Industry Id: ", clist['C'+str(i)].value, "             Tag: ", clist['D'+str(i)].value)
                cmd = input('What would you like to do?  (d)elete   (ns)not sure   (o)ther   (k)eep  (s)ave file\n')
                if (cmd == 's'):
                    file.save('/Users/wandi/Downloads/Leftover.xlsx')
                    print("current copy saved to file\n")
                else:
                    while (cmd!="ns" and cmd !='k' and cmd!='o' and cmd!='d'):
                        cmd = input('What would you like to do?  (d)elete   (ns)not sure   (o)ther   (k)eep  (s)ave file')
                    process[cmd](i)
            i+=1
        tag_id = input("Add a tag? Press f to exit")
        while(tag_id!='f'):
            if ( int(tag_id)<572 or int(tag_id) >608):
                print("Tag id not in range (572,609)")
            else:
                process['n'](Id,tag_id,inserted)
                inserted+=1
            tag_id = input("Add a tag? Press f to exit")
    print ("all rows have been processed")
###################################################################################################################

In [ ]:
manageEntries(2)

Link to the crunchbase of: Dover : https://www.crunchbase.com/organization/dover#/entity
Current entries for Dover :

Row: 2              Industry Id:  590              Tag:  Hardware and Devices
What would you like to do?  (d)elete   (ns)not sure   (o)ther   (k)eep  (s)ave filelkj
